In [2]:
import Function as f
import numpy as np
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.layers import LSTM,Input,Dense
from tensorflow.keras.models import Sequential


In [3]:
def lstm(note1,note2,note3):
# define an activation function

    # timesteps and number of features 
    timesteps = 5
    n_features = 1

    # define a simple lstm for
    model = Sequential()
    model.add(Input(shape=(timesteps,n_features)))
    model.add(LSTM(note1,return_sequences=True))
    model.add(LSTM(note2,return_sequences=False))
    model.add(Dense(note3,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mae')
    
    return model

def ann(note1,note2,note3):
    
    n_features = 5
    
    model = Sequential()
    model.add(Input(shape=(n_features,)))
    model.add(Dense(note1,'relu'))
    model.add(Dense(note2,'relu'))
    model.add(Dense(note3,'relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mae')
    
    return model

def generator_price_lstm(x_train, y_train, x_val, y_val, x_test, y_test, date_test,y_scaler,
              model_parameter, epochs, batch_size, thred):
    # initialize
    acc = []

    for j in range(thred):
        
        y_true = np.array([])
        y_pred = np.array([])
        date = np.array([])
        model = lstm(note1 = model_parameter[0],
                     note2=model_parameter[1],
                     note3=model_parameter[2])
        for i in range(len(y_train)):
        
            
            model.fit(x_train[i], y_train[i], validation_data=(x_val[i], y_val[i]),
                      epochs=epochs, batch_size=batch_size,verbose=0)

            predition = model.predict(x_test[i])

            model.reset_states()
            y_true = np.append(y_true, y_test[i])
            y_pred = np.append(y_pred, predition)
            date = np.append(date, date_test[i])
            

        y_true_transform = y_true * y_scaler[1] + y_scaler[0]
        y_pred_transform = y_pred * y_scaler[1] + y_scaler[0]
        
        acc.append(mean_absolute_error(y_true_transform, y_pred_transform))
        
    acc_min = np.min(acc)
  
    return acc_min

def generator_price_ann(x_train, y_train, x_val, y_val, x_test, y_test, date_test,y_scaler,
              model_parameter, epochs, batch_size, thred):
    # initialize
    acc = []

    for j in range(thred):
        
        model = ann(note1 = model_parameter[0],
                     note2=model_parameter[1],
                     note3=model_parameter[2])
        
        y_true = np.array([])
        y_pred = np.array([])
        date = np.array([])

        for i in range(len(y_train)):
            model.fit(x_train[i], y_train[i], validation_data=(x_val[i], y_val[i]),
                      epochs=epochs, batch_size=batch_size,verbose=0)

            predition = model.predict(x_test[i])

            model.reset_states()
            y_true = np.append(y_true, y_test[i])
            y_pred = np.append(y_pred, predition)
            date = np.append(date, date_test[i])
            
        y_true_transform = y_true * y_scaler[1] + y_scaler[0]
        y_pred_transform = y_pred * y_scaler[1] + y_scaler[0]
        
        acc.append(mean_absolute_error(y_true_transform, y_pred_transform))
        # print(j,mean_absolute_error(y_true, y_pred))
        
    acc_min = np.min(acc)
  
    return acc_min


In [4]:
# LSTM
times = 8
names = ['AGG','DBC','VIX','VTI']

small = []
medium1 = []
medium2 = []
large = []
    
for i in names:
    
    
    # loading data 
    x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler = f.data_generator_lstm(name=i,timesteps=5,
                                                                train_len=100,val_len=10,test_len=5,forward_len=5)
    # model 8/4/2
    model = [8,4,2]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    small.append(acc)
    
    # model 32/16
    model = [32,16,8]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                               model,epochs=25,batch_size=50,thred=times)
    medium1.append(acc)
    
    # model 64/32
    model = [64,32,16]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    medium2.append(acc)
    
    # model 128/64
    model = [128,64,32]
    acc = generator_price_lstm(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    large.append(acc)

In [5]:
print(small)
print(medium1)
print(medium2)
print(large)

print(np.mean(small))
print(np.mean(medium1))
print(np.mean(medium2))
print(np.mean(large))
# 1.864068123257817
# 1.287479630961372
# 0.9085718958843285
# 0.8217149862388022

[1.5349665136012645, 1.4928552908469979, 3.6134182721018706, 19.376698885220563]
[0.726897784374149, 0.42590969730948686, 3.3624412687096945, 4.805504873474457]
[0.7093644121351067, 0.4826735296298294, 3.7135709816938998, 4.542741341565977]
[0.7795008108012499, 0.4540556507959029, 3.980215448886378, 4.233624671912515]
6.504484740442674
2.330188405966947
2.362087566256203
2.3618491455990114


In [6]:
# ANN
times = 8
names = ['AGG','DBC','VIX','VTI']

small = []
medium1 = []
medium2 = []
large = []
    
for i in names:
    
    # loading data 
    x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler= f.data_generator_ann(name=i,features=5,
                                                                train_len=100,val_len=10,test_len=5,forward_len=5)
    # model 8/4/2
    model = [8,4,2]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    small.append(acc)
    
    # model 32/16/8
    model = [32,16,8]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                                 model,epochs=25,batch_size=50,thred=times)
    medium1.append(acc)
    
    # model 64/32/16
    model = [64,32,16]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                              model,epochs=25,batch_size=50,thred=times)
    medium2.append(acc)
    
    # model 128/64/32
    model = [128,64,32]
    acc = generator_price_ann(x_train,y_train,x_val,y_val,x_test,y_test,date_test,y_scaler,
                            model,epochs=25,batch_size=50,thred=times)
    large.append(acc)

In [6]:
print(small)
print(medium1)
print(medium2)
print(large)

print(np.mean(small))
print(np.mean(medium1))
print(np.mean(medium2))
print(np.mean(large))

[4.408627891076656, 0.6233060507915018, 3.760108539545844, 13.05705959692306]
[0.6612603173692976, 0.47017800698487894, 3.411698052861373, 5.089295208785433]
[0.6845273535554061, 0.3347084886231804, 3.2266436059926558, 4.167611431857856]
[0.7001445958526761, 0.39689532441360165, 3.5390280126221922, 4.292207598996064]
5.462275519584265
2.408107896500246
2.1033727200072745
2.2320688829711335
